In [19]:
import sympy as sp
from sympy import SingularityFunction as mac

p, m0, q, ra, ma, rb, rd, rf, rg, c1, c2, c3 = sp.symbols(
    'P, M_0, q, R_A, M_A, R_B, R_D, R_F, R_G, C_1:4'
)
x, d, e, i, b, h = sp.symbols('x, d, E, I, b, h', positive=True)

unknowns = [c1, c2, ma, ra, rb, rd, rf, rg]

# Known values for later subs
knowns_q7 = {
    d: 250e-3,
    p: 1e3,
    m0: 0,
    q: 1e3,
    e: 210e9,
    b: 25e-3,
    h: 50e-3,
    i: 0,
}

# Distances
ab = 2 * d
ac = ab + d
ad = ac + 1.5 * d
ae = ad + 1.5 * d
af = ae + 2 * d
ag = af + 3 * d
l = ag + .5 * d
fl = 3.5 * d

# Calculating I
i_eq = b * h ** 3 / 12
knowns_q7[i] = i_eq.subs(knowns_q7)

# Functions
sx = sp.Function('V')(x)
mx = sp.Function('M')(x)
thx = sp.Function('theta')(x)
vx = sp.Function('v')(x)

# Force equilibrium
f_sum_eq = sp.Eq(rb + rd + rf + rg, ra + p + q * fl)
# Moment equilibrium
m_sum_eq = sp.Eq(
    rb * ab + rd * ad + rf * af + rg * ag,
    ma + p * ac + m0 + q * fl * (af + fl / 2)
)

display(f_sum_eq.subs(knowns_q7))
display(m_sum_eq.subs(knowns_q7))

# Moment equation using Macaulay's method
#  "mac" is a shorthand for the sp.SingularityFunction method, 
#  which is essentially  Macaulay bracket notation: 
#   <x - a>^n <---> mac(x, a, n)
m_eq = - ra * mac(x, 0, 1) + ma * mac(x, 0, 0) + rb * mac(x, ab, 1) \
            - p * mac(x, ac, 1) + rd * mac(x, ad, 1) + m0 * mac(x, ae, 0) \
                + rf * mac(x, af, 1) - q / 2 * mac(x, af, 2) + rg * mac(x, ag, 1)

# Shear force equation obtained by differentiating m_eq wrt x
s_eq = sp.diff(m_eq, x)
# Slope equation obtained from integrating m_eq once wrt x
th_eq = sp.integrate(m_eq / (e * i), x) + c1
# Deflection equation obtained from integrating m_eq twice wrt x
v_eq = sp.integrate(th_eq, x) + c2

display(sp.Eq(sx, s_eq))
display(sp.Eq(mx, m_eq))
display(sp.Eq(thx, th_eq))
display(sp.Eq(vx, v_eq))

# Boundary and continuity conditions
#  Substitute
v0_eq = v_eq.subs(x, 0).subs(knowns_q7)
th0_eq = th_eq.subs(x, 0).subs(knowns_q7)
vb_eq = v_eq.subs(x, ab).subs(knowns_q7)
vd_eq = v_eq.subs(x, ad).subs(knowns_q7)
vf_eq = v_eq.subs(x, af).subs(knowns_q7)
vg_eq = v_eq.subs(x, ag).subs(knowns_q7)
ml_eq = m_eq.subs(x, l).subs(knowns_q7)
sl_eq = s_eq.subs(x, l).subs(knowns_q7)

mat_a, mat_b = sp.linear_eq_to_matrix(
    [v0_eq, th0_eq, vb_eq, vd_eq, vf_eq, vg_eq, ml_eq, sl_eq],
    unknowns
)

Eq(R_B + R_D + R_F + R_G, R_A + 1875.0)

Eq(0.5*R_B + 1.125*R_D + 2.0*R_F + 2.75*R_G, M_A + 2882.8125)

Eq(V(x), M_0*SingularityFunction(x, 6.0*d, -1) - P*SingularityFunction(x, 3*d, 0) - R_A + R_B*SingularityFunction(x, 2*d, 0) + R_D*SingularityFunction(x, 4.5*d, 0) + R_F*SingularityFunction(x, 8.0*d, 0) + R_G*SingularityFunction(x, 11.0*d, 0) - q*SingularityFunction(x, 8.0*d, 1))

Eq(M(x), M_0*SingularityFunction(x, 6.0*d, 0) + M_A - P*SingularityFunction(x, 3*d, 1) - R_A*x + R_B*SingularityFunction(x, 2*d, 1) + R_D*SingularityFunction(x, 4.5*d, 1) + R_F*SingularityFunction(x, 8.0*d, 1) + R_G*SingularityFunction(x, 11.0*d, 1) - q*SingularityFunction(x, 8.0*d, 2)/2)

Eq(theta(x), C_1 + (M_0*SingularityFunction(x, 6.0*d, 1) + M_A*x - P*SingularityFunction(x, 3*d, 2)/2 - R_A*x**2/2 + R_B*SingularityFunction(x, 2*d, 2)/2 + R_D*SingularityFunction(x, 4.5*d, 2)/2 + R_F*SingularityFunction(x, 8.0*d, 2)/2 + R_G*SingularityFunction(x, 11.0*d, 2)/2 - q*SingularityFunction(x, 8.0*d, 3)/6)/(E*I))

Eq(v(x), C_1*x + C_2 + (M_0*SingularityFunction(x, 6.0*d, 2)/2 + M_A*x**2/2 - P*SingularityFunction(x, 3*d, 3)/6 - R_A*x**3/6 + R_B*SingularityFunction(x, 2*d, 3)/6 + R_D*SingularityFunction(x, 4.5*d, 3)/6 + R_F*SingularityFunction(x, 8.0*d, 3)/6 + R_G*SingularityFunction(x, 11.0*d, 3)/6 - q*SingularityFunction(x, 8.0*d, 4)/24)/(E*I))

In [20]:
# Solving the system of equations
solns = sp.linsolve(
    [mat_a, mat_b],
    unknowns
).args[0]
solns_dict = {sym: val.subs(knowns_q7) for sym, val in zip(unknowns, solns)}

# Results
for sym, val in solns_dict.items():
    display(sp.Eq(sym, round(val, 6)))

Eq(C_1, 0)

Eq(C_2, 0)

Eq(M_A, 33.999452)

Eq(R_A, 203.996711)

Eq(R_B, 876.949342)

Eq(R_D, 324.517293)

Eq(R_F, 399.936508)

Eq(R_G, 477.593567)